# FLight usa dojo

### Prérequis :
- Installer apache Spark
- Java 8 car spark est ecrit en scala et qu'il lui faut donc une JVM (java virtual machine).


In [ ]:
!apt install openjdk-8-jdk-headless -qq
!pip install pyspark
!pip install requests

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

#### Lire la donnée

On va utiliser `DataFrameReader` pour lire la donnée, ce dernier est lié à notre `SparkSession` et adapté pour lire de gros volume de donnée.

Construction du schema : Dans notre cas on veut faire deviner à spark notre schema de donnée. On va donc utiliser l'option : `schema inference`.

In [ ]:
import requests

# Télécharger le csv
url = 'https://raw.githubusercontent.com/ADataGuru/labs/main/modules/batch/spark/dojos/fligths_usa/data/2015-summary.csv'
r = requests.get(url, allow_redirects=True)

open('2015-summary.csv', 'wb').write(r.content)

In [3]:
from pyspark.shell import spark

flightData2015 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("2015-summary.csv")

Ce dataframe possède un ensemble de colonnes avec un nombre non spécifié de lignes. La raison pour laquelle le nombre de lignes n'est pas spécifié est que la lecture des données est une transformation, et elle est donc une opération `lazy` (dite paraisseuse). Spark ne jette un coup d'œil qu'à quelques lignes de données pour essayer de deviner les types de chaque colonne. La figure 2-7 illustre la lecture du fichier CSV dans un DataFrame, puis sa conversion en un tableau local ou une liste de lignes.

![reader](static/reader.png)

Si nous effectuons l'action take sur le DataFrame, nous pourrons voir les 3 premières lignes du DataFrame:

In [ ]:
flightData2015.take(3)

In [ ]:
flightData2015.sort("count").take(3)

In [ ]:
flightData2015.take(3)

> N'oubliez pas que le tri ne modifie pas le DataFrame.
> Nous utilisons sort comme une transformation qui renvoie un nouveau DataFrame en transformant le DataFrame précédent.
> Illustrons ce qui se passe lorsque nous appelons take sur ce DataFrame résultant.

![figure 2-8. Reading, sorting, and collecting a DataFrame](static/reading_sorting_collecting.png)

Rien ne se produit dans les données lorsque nous appelons sort parce qu'il s'agit simplement d'une transformation. Cependant, nous pouvons voir que Spark est en train de construire un plan pour savoir comment il va exécuter cette requête à travers le cluster en regardant le plan explain. Nous pouvons appeler explain sur n'importe quel objet DataFrame pour voir le lineage (lignée) du DataFrame (ou comment Spark va exécuter cette requête) :

#### Explains

In [ ]:
flightData2015.sort("count").explain()

Félicitations, vous venez de lire votre premier plan d'explication ! Les plans d'explication sont un peu obscurs, mais avec un peu de pratique, ils deviennent une seconde nature. Vous pouvez lire les plans d'explication de haut en bas, le haut étant le résultat final, et le bas étant la ou les sources de données. Dans ce cas, jetez un coup d'œil aux premiers mots-clés. Vous verrez trier, échanger, et FileScan. C'est parce que le tri de nos données est en fait une transformation large, car les lignes devront être comparées les unes aux autres. Ne vous inquiétez pas trop de comprendre tout ce qui concerne les plans d'explication à ce stade, ils peuvent juste être des outils utiles pour déboguer et améliorer vos connaissances à mesure que vous progressez avec Spark.

Maintenant, comme nous l'avons fait auparavant, nous pouvons spécifier une action pour lancer ce plan. Cependant, avant de faire cela, nous allons définir une configuration. Par défaut, lorsque nous effectuons un shuffle, Spark produit 200 partitions de shuffle. Réglons cette valeur à 5 pour réduire le nombre de partitions de sortie du shuffle :

In [10]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [11]:
flightData2015.sort("count").take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

La photo ci-dessous illustre cette opération. Remarquez qu'en plus des transformations logiques, nous incluons également le nombre de partitions physiques.

![Figure 2-9. The process of logical and physical DataFrame manipulation](./static/logical_and_physical_partition.png)

Le plan logique des transformations que nous construisons définit un lineage (lignée) pour le DataFrame de sorte qu'à tout moment, Spark sait comment recalculer n'importe quelle partition en effectuant toutes les opérations qu'il avait auparavant sur les mêmes données d'entrée. Cela se trouve au cœur du modèle de programmation de Spark - la programmation fonctionnelle où les mêmes entrées donnent toujours les mêmes sorties lorsque les transformations sur ces données restent constantes.

Nous ne manipulons pas les données physiques ; au lieu de cela, nous configurons les caractéristiques d'exécution physique par le biais de choses comme le paramètre shuffle partitions que nous avons défini il y a quelques instants. Nous nous sommes retrouvés avec cinq partitions de sortie parce que c'est la valeur que nous avons spécifiée dans le paramètre shuffle partition. Vous pouvez modifier ce paramètre pour aider à contrôler les caractéristiques d'exécution physique de vos travaux Spark. Allez-y, expérimentez avec différentes valeurs et voyez vous-même le nombre de partitions. En expérimentant avec différentes valeurs, vous devriez voir des temps d'exécution radicalement différents. N'oubliez pas que vous pouvez surveiller la progression des travaux en naviguant vers l'interface utilisateur Spark sur le port 4040 pour voir les caractéristiques d'exécution physique et logique de vos travaux.

#### DataFrames and SQL

Nous avons travaillé sur une transformation simple dans l'exemple précédent, travaillons maintenant sur une transformation plus complexe et suivons-la à la fois dans DataFrames et SQL. Spark peut exécuter les mêmes transformations, quel que soit le langage, exactement de la même manière. Vous pouvez exprimer votre logique d'entreprise en SQL ou DataFrames (soit en R, Python, Scala ou Java) et Spark compilera cette logique vers un plan sous-jacent (que vous pouvez voir dans le plan d'explication) avant d'exécuter réellement votre code. Avec Spark SQL, vous pouvez enregistrer n'importe quel DataFrame comme une table ou une vue (une table temporaire) et l'interroger en utilisant du pur SQL. Il n'y a pas de différence de performance entre l'écriture de requêtes SQL ou l'écriture de code DataFrame, ils sont tous deux "compilés" au même plan sous-jacent que nous spécifions dans le code DataFrame.

Vous pouvez transformer n'importe quel DataFrame en tableau ou en vue par un simple appel de méthode :

In [12]:
flightData2015.createOrReplaceTempView("flight_data_2015")

Maintenant, nous pouvons interroger nos données en SQL. Pour ce faire, nous allons utiliser la fonction spark.sql (rappelez-vous, spark est notre variable SparkSession) qui renvoie commodément un nouveau DataFrame. Bien que cela puisse sembler un peu circulaire dans la logique - qu'une requête SQL contre un DataFrame renvoie un autre DataFrame - c'est en fait assez puissant. Cela vous permet de spécifier les transformations de la manière qui vous convient le mieux à un moment donné et de ne pas sacrifier l'efficacité pour le faire ! Pour comprendre ce qui se passe, examinons deux plans d'explication :

In [13]:
# in Python
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

dataFrameWay = flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .count()

sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#40], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#40, 5), ENSURE_REQUIREMENTS, [id=#74]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#40], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#40] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/loic.caminale/Workspace/formation/dataguru/labs/modules/ba..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#40], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#40, 5), ENSURE_REQUIREMENTS, [id=#87]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#40], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#40] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIn

Tirons quelques statistiques intéressantes de nos données. Une chose à comprendre est que les DataFrames (et SQL) dans Spark ont déjà un nombre énorme de manipulations disponibles. Il existe des centaines de fonctions que vous pouvez utiliser et importer pour vous aider à résoudre plus rapidement vos problèmes de big data. Nous allons utiliser la fonction max, pour établir le nombre maximum de vols à destination et en provenance d'un lieu donné. Cette fonction analyse simplement chaque valeur de la colonne concernée dans le DataFrame et vérifie si elle est supérieure aux valeurs précédentes. Il s'agit d'une transformation, car nous filtrons effectivement jusqu'à une ligne. Voyons à quoi cela ressemble :

In [19]:
spark.sql("SELECT max(count) from flight_data_2015").take(1)

[Row(max(count)=370002)]

In [18]:
from pyspark.sql.functions import max

flightData2015.select(max("count")).take(1)

[Row(max(count)=370002)]

Super, c'est un exemple simple qui donne un résultat de 370 002. Effectuons quelque chose d'un peu plus compliqué et trouvons les cinq principaux pays de destination dans les données. Il s'agit de notre première requête de multi-transformation, nous allons donc procéder étape par étape. Commençons par une agrégation SQL assez simple :

In [21]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



Passons maintenant à la syntaxe DataFrame qui est sémantiquement similaire mais légèrement différente dans son implémentation et son ordonnancement. Mais, comme nous l'avons mentionné, les plans sous-jacents sont les mêmes pour les deux. Exécutons les requêtes et voyons leurs résultats pour vérifier qu'elles sont correctes :

In [22]:
from pyspark.sql.functions import desc

flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



Il y a maintenant sept étapes qui nous ramènent aux données sources. Vous pouvez le constater dans le plan d'explication de ces DataFrames. La figure ci-dessous montre l'ensemble des étapes que nous exécutons dans le "code". Le véritable plan d'exécution (celui qui est visible dans explain) sera différent de celui illustré ci-dessous en raison des optimisations de l'exécution physique ; toutefois, l'illustration est un bon point de départ. Ce plan d'exécution est un graphe acyclique dirigé (DAG) de transformations, chacune résultant en un nouveau DataFrame immuable, sur lequel nous appelons une action pour générer un résultat.

![Figure 2-10. The entire DataFrame transformation flow
](static/entire_transformation_flow.png)

La première étape consiste à lire les données. Nous avons défini le DataFrame précédemment mais, pour rappel, Spark ne le lit pas réellement jusqu'à ce qu'une action soit appelée sur ce DataFrame ou sur un dérivé du DataFrame original.

La deuxième étape est notre regroupement ; techniquement, lorsque nous appelons groupBy, nous obtenons un RelationalGroupedDataset, qui est un nom fantaisiste pour un DataFrame qui a un regroupement spécifié mais qui a besoin que l'utilisateur spécifie une agrégation avant de pouvoir être interrogé. En gros, nous avons spécifié que nous allions regrouper les données par une clé (ou un ensemble de clés) et que nous allions maintenant effectuer une agrégation sur chacune de ces clés.

Par conséquent, la troisième étape consiste à spécifier l'agrégation. Utilisons la méthode d'agrégation par la somme. Celle-ci prend en entrée une expression de colonne ou, simplement, un nom de colonne. Le résultat de l'appel de la méthode sum est un nouveau DataFrame. Vous verrez qu'il a un nouveau schéma mais qu'il connaît le type de chaque colonne. Il est important de souligner (encore une fois !) qu'aucun calcul n'a été effectué. Il s'agit simplement d'une autre transformation que nous avons exprimée, et Spark est simplement capable de suivre nos informations de type à travers elle.


La quatrième étape est un simple renommage. Nous utilisons la méthode withColumnRenamed qui prend deux arguments, le nom original de la colonne et le nouveau nom de la colonne. Bien entendu, cette méthode n'effectue aucun calcul : il s'agit simplement d'une transformation supplémentaire !

La cinquième étape trie les données de telle sorte que si nous prenions les résultats en haut du DataFrame, ils auraient les plus grandes valeurs dans la colonne destination_total.

Enfin, nous allons spécifier une limite. Ceci spécifie simplement que nous voulons seulement retourner les cinq premières valeurs dans notre DataFrame final au lieu de toutes les données.

La dernière étape est notre action ! Maintenant, nous commençons réellement le processus de collecte des résultats de notre DataFrame, et Spark nous rendra une liste ou un tableau dans le langage que nous exécutons. Pour renforcer tout cela, regardons le plan d'explication de la requête précédente :

In [24]:
# in Python
flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#160L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#40,destination_total#160L])
   +- HashAggregate(keys=[DEST_COUNTRY_NAME#40], functions=[sum(count#42)])
      +- Exchange hashpartitioning(DEST_COUNTRY_NAME#40, 5), ENSURE_REQUIREMENTS, [id=#383]
         +- HashAggregate(keys=[DEST_COUNTRY_NAME#40], functions=[partial_sum(count#42)])
            +- FileScan csv [DEST_COUNTRY_NAME#40,count#42] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/loic.caminale/Workspace/formation/dataguru/labs/modules/ba..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>




Bien que ce plan d'explication ne corresponde pas exactement à notre "plan conceptuel", tous les éléments sont là. Vous pouvez voir l'instruction limit ainsi que le orderBy (dans la première ligne). Vous pouvez également voir comment notre agrégation se fait en deux phases, dans les appels partial_sum. C'est parce que la somme d'une liste de nombres est commutative, et Spark peut effectuer la somme, partition par partition. Bien sûr, nous pouvons voir comment nous lisons dans le DataFrame, ainsi.

Naturellement, nous n'avons pas toujours besoin de collecter les données. Nous pouvons également les écrire vers n'importe quelle source de données prise en charge par Spark. Par exemple, supposons que nous voulions stocker les informations dans une base de données comme PostgreSQL ou les écrire dans un autre fichier.